In this section we are going to implement the liner regression model using pytorch.

### Import Libraries

In [ ]:
# import required libraries
import numpy as np
import torch
from torch.utils import data

### Generating Data

In [3]:
# generating synthetic data 

def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [9]:
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([ 0.7717, -0.0837]) 
label: tensor([6.0404])


### Reading the Dataset :

- Rather than rolling our own iterator, we can call upon the existing API in a framework to read data. We pass in `features` and `labels` as arguments and specify `batch_size` when instantiating a data iterator object. 

- boolean value `is_train` indicates whether or not we want the data iterator object to shuffle the data on each epoch.


In [10]:
def load_array(data_arrays, batch_size, is_train=True):
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

Now we can use `data_iter` in much the same way as we called the `data_iter` function in scratch implementation.

To verify that it is working, we can read and print the first minibatch of examples. 

Here we use `iter` to construct a Python iterator and use `next` to obtain the first item from the iterator.

In [11]:
next(iter(data_iter))

[tensor([[ 1.3855,  0.6026],
         [-1.1590, -1.1072],
         [ 0.3469, -1.0460],
         [-0.4812, -0.7850],
         [-1.3633, -0.2921],
         [ 0.4584,  0.4201],
         [ 0.2525, -0.7704],
         [ 0.8398,  1.3273],
         [ 0.0024,  1.7252],
         [ 0.9644, -0.5296]]), tensor([[ 4.9029],
         [ 5.6415],
         [ 8.4601],
         [ 5.9251],
         [ 2.4735],
         [ 3.6923],
         [ 7.3126],
         [ 1.3502],
         [-1.6613],
         [ 7.9276]])]

### Defining the model :

When we implemented linear regression from scratch, we defined our model parameters explicitly and coded up the calculations to produce output using basic linear algebra operations. 

You should know how to do this. But once your models get more complex, and once you have to do this nearly every day, you will be glad for the assistance.

`Blog Examaple`

For standard operations, we can use a framework’s predefined layers, which allow us to focus especially on the layers used to construct the model rather than having to focus on the implementation.



- We will first define a model variable `net`, which will refer to an instance of the Sequential class.

- Given input data, a Sequential instance passes it through the first layer, in turn passing the output as the second layer’s input and so forth.

In [12]:
# `nn` -- > neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In PyTorch, the fully-connected layer is defined in the Linear class. Note that we passed two arguments into `nn.Linear`. The first one specifies the input feature dimension, which is 2, and the second one is the output feature dimension, which is a single scalar and therefore 1.

### Inintializing model parameter : 

Before using net, we need to initialize the model parameters, such as the weights and bias in the linear regression model. Deep learning frameworks often have a predefined way to initialize the parameters. Here we specify that each weight parameter should be randomly sampled from a normal distribution with mean 0 and standard deviation 0.01. The bias parameter will be initialized to zero.

As we have specified the input and output dimensions when constructing `nn.Linear`, now we can access the parameters directly to specify their initial values.

- We first locate the layer by `net[0]`, which is the first layer in the network
- and then use the `weight.data` and `bias.data` methods to access the parameters
- Next we use the replace methods `normal_` and `fill_` to overwrite parameter values.

In [13]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### Defining the loss function 

The MSELoss class computes the mean squared error. By default it returns the average loss over examples.

In [14]:
loss = nn.MSELoss()

### Defining Optimization Algorithm : 

Minibatch stochastic gradient descent is a standard tool for optimizing neural networks and thus PyTorch supports it alongside a number of variations on this algorithm in the `optim` module.

When we instantiate an SGD instance, we will specify the parameters to optimize over (obtainable from our net via `net.parameters()`), with a dictionary of hyperparameters required by our optimization algorithm.

Minibatch stochastic gradient descent just requires that we set the value `lr`, which is set to 0.03 here.

In [15]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

### Training : 

You might have noticed that expressing our model through high-level APIs of a deep learning framework requires comparatively few lines of code. We did not have to individually allocate parameters, define our loss function, or implement minibatch stochastic gradient descent. Once we start working with much more complex models, advantages of high-level APIs will grow considerably. However, once we have all the basic pieces in place, the training loop itself is strikingly similar to what we did when implementing everything from scratch.

To refresh your memory: for some number of epochs, we will make a complete pass over the dataset (train_data), iteratively grabbing one minibatch of inputs and the corresponding ground-truth labels. For each minibatch, we go through the following ritual:

- Generate predictions by calling net(X) and calculate the loss l (the forward propagation).

- Calculate gradients by running the backpropagation.

- Update the model parameters by invoking our optimizer.

For good measure, we compute the loss after each epoch and print it to monitor progress.

In [16]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000234
epoch 2, loss 0.000110
epoch 3, loss 0.000108


Below, we compare the model parameters learned by training on finite data and the actual parameters that generated our dataset. To access parameters, we first access the layer that we need from net and then access that layer’s weights and bias.

In [17]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-0.0011, -0.0005])
error in estimating b: tensor([0.0002])


- Using PyTorch’s high-level APIs, we can implement models much more concisely.

- In PyTorch, the data module provides tools for data processing, the nn module defines a large number of neural network layers and common loss functions.

- We can initialize the parameters by replacing their values with methods ending with _.